In [1]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import torch

# ==================== КОНФИГУРАЦИЈА ====================
CURRENT_DIR = os.getcwd()
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, "..", ".."))
DATA_YAML = os.path.join(BASE_DIR, "datasets", "coin", "data.yaml")

print("=" * 70)
print("🪙 МАКЕДОНСКИ ДЕТЕКТОР ЗА МОНЕТИ - Оптимизирана верзија")
print("=" * 70)
print(f"📁 BASE_DIR: {BASE_DIR}")
print(f"📄 DATA_YAML: {DATA_YAML}")
print("=" * 70)


# ==================== КОНФИГУРАЦИЈА ЗА ТРЕНИРАЊЕ ====================
class CoinDetectorConfig:
    """Конфигурација оптимизирана за мал датасет (70 слики по класа)"""

    # Основни параметри
    TRAIN_NEW = True
    MODEL_SIZE = 'yolov8m.pt'  # Medium модел - подобар од 's' за precision

    # Тренирање параметри
    EPOCHS = 200  # Повеќе епохи за мал датасет
    BATCH_SIZE = 8  # Помал batch за подобра генерализација
    IMAGE_SIZE = 640
    PATIENCE = 50  # Долго трпение за мал датасет

    # Optimizer
    OPTIMIZER = "AdamW"
    LR0 = 0.0005  # Помала learning rate за fine-tuning
    LRF = 0.001
    MOMENTUM = 0.937
    WEIGHT_DECAY = 0.001  # Повисок за да се спречи overfitting
    WARMUP_EPOCHS = 5.0

    # Аугментација (AGGRESSIVE за мал датасет)
    HSV_H = 0.02  # Варијација на боја
    HSV_S = 0.8
    HSV_V = 0.5
    DEGREES = 45  # Зголемена ротација
    TRANSLATE = 0.2
    SCALE = 0.7
    SHEAR = 5.0
    PERSPECTIVE = 0.001
    FLIPUD = 0.5
    FLIPLR = 0.5
    MOSAIC = 1.0
    MIXUP = 0.15  # Додадено mixup за подобра генерализација
    COPY_PASTE = 0.3  # Copy-paste аугментација

    # Loss weights (оптимизирано за монети)
    BOX = 7.5
    CLS = 1.0  # Зголемено за подобра класификација
    DFL = 1.5

    # Inference
    CONF_THRESHOLD = 0.35  # Поникок threshold за подобра recall
    IOU_THRESHOLD = 0.5

    # Output
    MODEL_NAME = 'coin_detector_mk_optimized'
    SAVE_PERIOD = 20

    @classmethod
    def get_model_path(cls):
        return os.path.join(BASE_DIR, "models", cls.MODEL_NAME, "weights", "best.pt")


# ==================== ФУНКЦИИ ЗА ОБРАБОТКА НА СЛИКИ ====================
def remove_background_grabcut(image, bbox, padding=5):
    """
    GrabCut метод - одличен за монети со комплексна позадина

    Args:
        image: Влезна слика
        bbox: Bounding box [x1, y1, x2, y2]
        padding: Padding околу bbox
    """
    x1, y1, x2, y2 = map(int, bbox)
    h, w = image.shape[:2]

    # Додај padding
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(w, x2 + padding)
    y2 = min(h, y2 + padding)

    cropped = image[y1:y2, x1:x2].copy()

    if cropped.size == 0 or cropped.shape[0] < 10 or cropped.shape[1] < 10:
        return np.zeros((1, 1, 4), dtype=np.uint8)

    # GrabCut rectangle (внатрешен, помал од целиот crop)
    ch, cw = cropped.shape[:2]
    rect = (int(cw*0.1), int(ch*0.1), int(cw*0.8), int(ch*0.8))

    # Иницijализирај маски
    mask = np.zeros(cropped.shape[:2], np.uint8)
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)

    try:
        # GrabCut алгоритам
        cv2.grabCut(cropped, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)

        # Маска за foreground
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

        # Морфолошки операции
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_OPEN, kernel)

        # Gaussian blur за smooth edges
        mask2 = (mask2 * 255).astype(np.uint8)
        mask2 = cv2.GaussianBlur(mask2, (7, 7), 0)

    except:
        # Fallback ако GrabCut не успее
        mask2 = np.ones(cropped.shape[:2], dtype=np.uint8) * 255

    # Креирај BGRA
    bgra = cv2.cvtColor(cropped, cv2.COLOR_BGR2BGRA)
    bgra[:, :, 3] = mask2

    return bgra


def remove_background_edge_detection(image, bbox, padding=10):
    """
    Edge detection метод - одличен за монети со остар раб

    Args:
        image: Влезна слика
        bbox: Bounding box [x1, y1, x2, y2]
        padding: Padding
    """
    x1, y1, x2, y2 = map(int, bbox)
    h, w = image.shape[:2]

    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(w, x2 + padding)
    y2 = min(h, y2 + padding)

    cropped = image[y1:y2, x1:x2].copy()

    if cropped.size == 0 or cropped.shape[0] == 0 or cropped.shape[1] == 0:
        return np.zeros((1, 1, 4), dtype=np.uint8)

    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    # Bilateral filter - зачува рабови
    filtered = cv2.bilateralFilter(gray, 9, 75, 75)

    # Canny edge detection
    edges = cv2.Canny(filtered, 30, 100)

    # Dilate за да се затворат празнините
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros(cropped.shape[:2], dtype=np.uint8)

    if contours:
        # Најди најголем contour (претпоставка: тоа е монетата)
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest_contour], -1, 255, -1)
    else:
        # Fallback: цела слика
        mask = np.ones(cropped.shape[:2], dtype=np.uint8) * 255

    # Морфолошки операции
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.GaussianBlur(mask, (9, 9), 0)

    bgra = cv2.cvtColor(cropped, cv2.COLOR_BGR2BGRA)
    bgra[:, :, 3] = mask

    return bgra


def remove_background_hough_circles(image, bbox, padding=10):
    """
    Hough Circles метод - оригиналниот, подобрен

    Args:
        image: Влезна слика
        bbox: Bounding box [x1, y1, x2, y2]
        padding: Padding околу монетата
    """
    x1, y1, x2, y2 = map(int, bbox)
    h, w = image.shape[:2]

    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(w, x2 + padding)
    y2 = min(h, y2 + padding)

    cropped = image[y1:y2, x1:x2].copy()

    if cropped.size == 0 or cropped.shape[0] == 0 or cropped.shape[1] == 0:
        return np.zeros((1, 1, 4), dtype=np.uint8)

    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(blurred)

    # Hough Circles со повеќе attempts
    circles = None
    params = [
        {'param1': 50, 'param2': 30, 'minR': 0.20, 'maxR': 0.70},
        {'param1': 40, 'param2': 25, 'minR': 0.25, 'maxR': 0.65},
        {'param1': 60, 'param2': 35, 'minR': 0.15, 'maxR': 0.75},
    ]

    for param in params:
        circles = cv2.HoughCircles(
            enhanced,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=max(cropped.shape[:2]) // 2,
            param1=param['param1'],
            param2=param['param2'],
            minRadius=int(min(cropped.shape[:2]) * param['minR']),
            maxRadius=int(max(cropped.shape[:2]) * param['maxR'])
        )
        if circles is not None:
            break

    mask = np.zeros(cropped.shape[:2], dtype=np.uint8)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        # Најголема кружница
        largest_circle = max(circles[0], key=lambda c: c[2])
        center = (largest_circle[0], largest_circle[1])
        radius = int(largest_circle[2] * 1.02)
        cv2.circle(mask, center, radius, 255, -1)
    else:
        # Fallback: елипса базирана на bbox
        ch, cw = cropped.shape[:2]
        center = (cw // 2, ch // 2)
        axes = (int(cw * 0.47), int(ch * 0.47))
        cv2.ellipse(mask, center, axes, 0, 0, 360, 255, -1)

    # Морфолошки операции
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.GaussianBlur(mask, (11, 11), 0)

    bgra = cv2.cvtColor(cropped, cv2.COLOR_BGR2BGRA)
    bgra[:, :, 3] = mask

    return bgra


def remove_background_hybrid(image, bbox, padding=10, method='auto'):
    """
    Хибриден метод - комбинира најдобри техники

    Args:
        image: Влезна слика
        bbox: Bounding box
        padding: Padding
        method: 'auto', 'grabcut', 'hough', 'edge', или 'ensemble'

    Returns:
        BGRA слика со transparent позадина
    """
    if method == 'auto':
        # Автоматски избор базиран на bbox големина
        x1, y1, x2, y2 = map(int, bbox)
        area = (x2 - x1) * (y2 - y1)

        if area > 50000:  # Голема монета
            method = 'grabcut'
        elif area < 10000:  # Мала монета
            method = 'hough'
        else:
            method = 'edge'

    if method == 'grabcut':
        return remove_background_grabcut(image, bbox, padding)
    elif method == 'hough':
        return remove_background_hough_circles(image, bbox, padding)
    elif method == 'edge':
        return remove_background_edge_detection(image, bbox, padding)
    elif method == 'ensemble':
        # Комбинирај сите три методи
        mask1 = remove_background_grabcut(image, bbox, padding)[:, :, 3]
        mask2 = remove_background_hough_circles(image, bbox, padding)[:, :, 3]
        mask3 = remove_background_edge_detection(image, bbox, padding)[:, :, 3]

        # Weighted average
        combined_mask = (mask1 * 0.4 + mask2 * 0.4 + mask3 * 0.2).astype(np.uint8)

        x1, y1, x2, y2 = map(int, bbox)
        h, w = image.shape[:2]
        x1 = max(0, x1 - padding)
        y1 = max(0, y1 - padding)
        x2 = min(w, x2 + padding)
        y2 = min(h, y2 + padding)
        cropped = image[y1:y2, x1:x2].copy()

        bgra = cv2.cvtColor(cropped, cv2.COLOR_BGR2BGRA)
        bgra[:, :, 3] = combined_mask

        return bgra
    else:
        # Default: Hough
        return remove_background_hough_circles(image, bbox, padding)


def enhance_coin_image(coin_bgra):
    """Подобри визуелен квалитет на монетата"""
    # Екстрахирај каналите
    b, g, r, a = cv2.split(coin_bgra)

    # CLAHE за подобар контраст
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    b = clahe.apply(b)
    g = clahe.apply(g)
    r = clahe.apply(r)

    # Спојги назад
    enhanced = cv2.merge([b, g, r, a])

    return enhanced


# ==================== ТРЕНИРАЊЕ ====================
def train_coin_detector(config=CoinDetectorConfig):
    """Тренирај модел со оптимизирани параметри"""

    print("\n" + "=" * 70)
    print("🏋️  ЗАПОЧНУВА ТРЕНИРАЊЕ НА МОДЕЛОТ")
    print("=" * 70)
    print(f"📊 Модел: {config.MODEL_SIZE}")
    print(f"📈 Епохи: {config.EPOCHS}")
    print(f"📦 Batch size: {config.BATCH_SIZE}")
    print(f"🔍 Image size: {config.IMAGE_SIZE}")
    print(f"🎯 Optimizer: {config.OPTIMIZER}")
    print("=" * 70 + "\n")

    # Вчитај претходно обучен модел
    model = YOLO(config.MODEL_SIZE)

    # Тренирај
    results = model.train(
        data=DATA_YAML,
        epochs=config.EPOCHS,
        imgsz=config.IMAGE_SIZE,
        batch=config.BATCH_SIZE,
        patience=config.PATIENCE,
        project=os.path.join(BASE_DIR, "models"),
        name=config.MODEL_NAME,
        device=0 if torch.cuda.is_available() else 'cpu',

        # Optimizer
        optimizer=config.OPTIMIZER,
        lr0=config.LR0,
        lrf=config.LRF,
        momentum=config.MOMENTUM,
        weight_decay=config.WEIGHT_DECAY,
        warmup_epochs=config.WARMUP_EPOCHS,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,

        # Аугментација
        hsv_h=config.HSV_H,
        hsv_s=config.HSV_S,
        hsv_v=config.HSV_V,
        degrees=config.DEGREES,
        translate=config.TRANSLATE,
        scale=config.SCALE,
        shear=config.SHEAR,
        perspective=config.PERSPECTIVE,
        flipud=config.FLIPUD,
        fliplr=config.FLIPLR,
        mosaic=config.MOSAIC,
        mixup=config.MIXUP,
        copy_paste=config.COPY_PASTE,

        # Loss weights
        box=config.BOX,
        cls=config.CLS,
        dfl=config.DFL,

        # Output
        save=True,
        save_period=config.SAVE_PERIOD,
        plots=True,
        verbose=True,

        # Дополнителни параметри за подобри резултати
        amp=True,  # Automatic Mixed Precision
        fraction=1.0,  # Користи цел датасет
        close_mosaic=10,  # Исклучи mosaic последните 10 епохи
    )

    print("\n" + "=" * 70)
    print("✅ ТРЕНИРАЊЕТО ЗАВРШИ УСПЕШНО!")
    print("=" * 70)
    print(f"📁 Модел зачуван во: {config.get_model_path()}")
    print(f"📊 Графикони достапни во: {os.path.join(BASE_DIR, 'models', config.MODEL_NAME)}")
    print("=" * 70 + "\n")

    return model


# ==================== ВАЛИДАЦИЈА И ТЕСТИРАЊЕ ====================
def validate_model(model, verbose=True):
    """Детална валидација на моделот"""
    print("\n" + "=" * 70)
    print("📊 ВАЛИДАЦИЈА НА МОДЕЛОТ")
    print("=" * 70)

    metrics = model.val()

    if verbose:
        print(f"\n{'Метрика':<20} {'Вредност':>10}")
        print("-" * 32)
        print(f"{'mAP50':<20} {metrics.box.map50:>10.4f}")
        print(f"{'mAP50-95':<20} {metrics.box.map:>10.4f}")
        print(f"{'Precision':<20} {metrics.box.mp:>10.4f}")
        print(f"{'Recall':<20} {metrics.box.mr:>10.4f}")
        print(f"{'F1-Score':<20} {2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr + 1e-6):>10.4f}")
        print("-" * 32)

        # Per-class метрики
        if hasattr(metrics.box, 'ap_class_index') and len(metrics.box.ap_class_index) > 0:
            print(f"\n{'Класа':<15} {'AP50':>10} {'Precision':>10} {'Recall':>10}")
            print("-" * 47)
            for i, cls_idx in enumerate(metrics.box.ap_class_index):
                class_name = model.names[int(cls_idx)]
                print(f"{class_name:<15} {metrics.box.ap50[i]:>10.4f} {metrics.box.p[i]:>10.4f} {metrics.box.r[i]:>10.4f}")

    print("=" * 70 + "\n")
    return metrics


def test_detection(model, image_path, save_output=True, config=CoinDetectorConfig):
    """
    Тестирај детекција со зачувување на процесирани монети

    Args:
        model: YOLO модел
        image_path: Патека до слика
        save_output: Дали да се зачуваат резултатите
        config: Конфигурација
    """
    if not os.path.exists(image_path):
        print(f"❌ Сликата не постои: {image_path}")
        return None

    print(f"\n🔍 Детектирање монети во: {os.path.basename(image_path)}")
    image = cv2.imread(image_path)

    if image is None:
        print(f"❌ Грешка при вчитување на сликата")
        return None

    # Детекција
    results = model.predict(
        image,
        conf=config.CONF_THRESHOLD,
        iou=config.IOU_THRESHOLD,
        verbose=False
    )

    detected_coins = []

    for result in results:
        boxes = result.boxes

        if len(boxes) == 0:
            print("   ⚠️  Не се детектирани монети")
            return []

        print(f"\n   ✅ Пронајдени {len(boxes)} монети:")
        print(f"   {'#':<4} {'Класа':<12} {'Confidence':<12} {'Bbox'}")
        print("   " + "-" * 60)

        for i, box in enumerate(boxes):
            bbox = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0].cpu().numpy())
            cls = int(box.cls[0].cpu().numpy())
            class_name = model.names[cls]

            print(f"   {i+1:<4} {class_name:<12} {conf:>6.2%}      {bbox.astype(int).tolist()}")

            # Отстрани позадина - избери метод
            # Опции: 'auto', 'grabcut', 'hough', 'edge', 'ensemble'
            coin_bgra = remove_background_hybrid(image, bbox, padding=10, method='auto')
            coin_enhanced = enhance_coin_image(coin_bgra)

            detected_coins.append({
                'class': class_name,
                'confidence': conf,
                'bbox': bbox,
                'image': coin_bgra,
                'image_enhanced': coin_enhanced
            })

            # Зачувај
            if save_output:
                output_dir = os.path.join(BASE_DIR, "output", "detected_coins")
                os.makedirs(output_dir, exist_ok=True)

                # Оригинал
                orig_path = os.path.join(output_dir, f"{i+1}_{class_name}_{conf:.3f}.png")
                cv2.imwrite(orig_path, coin_bgra)

                # Enhanced
                enh_path = os.path.join(output_dir, f"{i+1}_{class_name}_{conf:.3f}_enhanced.png")
                cv2.imwrite(enh_path, coin_enhanced)

    if save_output and len(detected_coins) > 0:
        print(f"\n   💾 Зачувано {len(detected_coins) * 2} слики во: {output_dir}")

    return detected_coins


# ==================== ГЛАВНА ЛОГИКА ====================
def main():
    """Главна функција"""
    config = CoinDetectorConfig

    if config.TRAIN_NEW:
        # Тренирај нов модел
        model = train_coin_detector(config)
        model_path = config.get_model_path()
    else:
        # Вчитај постоечки
        model_path = config.get_model_path()

        if not os.path.exists(model_path):
            print(f"\n❌ Моделот не постои: {model_path}")
            print("💡 Промени CoinDetectorConfig.TRAIN_NEW = True за тренирање\n")
            return None

        print(f"\n📂 Вчитување модел од: {model_path}")
        model = YOLO(model_path)
        print("✅ Моделот е успешно вчитан!\n")

    # Прикажи информации
    print("📋 Информации за моделот:")
    print(f"   Број на класи: {len(model.names)}")
    print(f"   Класи: {list(model.names.values())}")

    # Валидирај
    if os.path.exists(DATA_YAML):
        validate_model(model)

    print("\n" + "=" * 70)
    print("🎯 МОДЕЛОТ Е ПОДГОТВЕН ЗА УПОТРЕБА!")
    print("=" * 70)
    print("\n💡 Примери за употреба:")
    print("   # Тестирај детекција")
    print("   test_detection(model, 'path/to/image.jpg')")
    print("\n   # Валидирај модел")
    print("   validate_model(model)")
    print("\n   # Директна предикција")
    print("   results = model.predict('path/to/image.jpg', conf=0.35)")
    print("=" * 70 + "\n")

    return model


# Стартувај
if __name__ == "__main__":
    model = main()

🪙 МАКЕДОНСКИ ДЕТЕКТОР ЗА МОНЕТИ - Оптимизирана верзија
📁 BASE_DIR: C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training
📄 DATA_YAML: C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training\datasets\coin\data.yaml

🏋️  ЗАПОЧНУВА ТРЕНИРАЊЕ НА МОДЕЛОТ
📊 Модел: yolov8m.pt
📈 Епохи: 200
📦 Batch size: 8
🔍 Image size: 640
🎯 Optimizer: AdamW

New https://pypi.org/project/ultralytics/8.4.6 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.241  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=1.0, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training\datasets\coin\data.yaml, degrees

C:\Users\User5549\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Model summary: 169 layers, 25,859,215 parameters, 25,859,199 gradients, 79.1 GFLOPs

Transferred 469/475 items from pretrained weights


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
